# Simulations for the paper "Empirical Bernstein and betting confidence intervals for randomized quasi-Monte Carlo"

To use Randomized Quasi-Monte Carlo (RQMC) for empirical Bernstein confidence
intervals (EBCI) and hedged betting confidence intervals (HBCI) described in [Estimating means of bounded random variables by betting](https://academic.oup.com/jrsssb/article/86/1/1/7043257) by Ian Waudby-Smith and Aaditya Ramdas, we take

$$
Y_i = \frac{1}{n}\sum_{k=1}^n f(\mathbf{x}_{i,k}), \qquad i=1,\dots,R
$$
where $f$ is any function bounded between $[0,1]$ and for each $i$, $\mathbf{x}_{i,1}, \dots, \mathbf{x}_{i,n}$ is an RQMC
point set. The $Y_i$ are IID when we use independent
scrambles for all $R$ point sets. The total budget is $N = nR$ where $R$ is the number of IID replications and $n$ is the number of RQMC points/evaluations per replication.


In this notebook, we use two one-dimensional integrands: 

$Y = f(X) = \frac{X e^X}{e}$

$ Y = f(X) = \begin{cases} 
1 & \text{if } X < \frac{1}{3} \\
0 & \text{otherwise}
\end{cases}$

We also use the following multi-dimensional ridge functions:

1. $ g_{jmp}(w) = 1{\{w \geq 1\}} $
2. $ g_{knk}(w) = \frac {\min(\max(−2, w), 1) + 2} {3} $
3. $ g_{smo}(w) = \Phi (w)$
4. $ g_{fin}(w) = \min(1,\sqrt{\max(w + 2, 0)}/2) $

$w = \frac{1}{\sqrt{d}} \sum_{j=1}^{d}\Phi^{-1}(x_{j})$, $\Phi(.)$ is the CDF of standard Normal Distribution, denoted by $ \mathcal{N}(0,1)$, and $\mathbf{x} \sim U[0, 1]^d$.

We have used DigitalNetB2 (also known as Sobol) Linear Matrix Scrambling + Digital Shift (LMS_DS) for RQMC.

Importing the necessary modules:

In [ ]:
import numpy as np
from scipy.stats import norm,t
from confseq.betting import betting_ci_seq
from confseq.predmix import predmix_empbern_ci_seq
import qmcpy as qp
import pandas as pd
import warnings
import contextlib

Supressing any warnings:

In [ ]:
@contextlib.contextmanager
def suppress_ci_warnings():
    with warnings.catch_warnings(), np.errstate(all="ignore"):
        warnings.simplefilter("ignore")
        yield

The parameters used for our numerical experiments

In [ ]:
alpha = 0.05 # Significance level, confidence level = 1 - alpha

# parameters used for the integrand problem

# The integrand functions:
fs = {
    "smooth_1d": lambda x: x[...,0]*np.exp(x[...,0])/np.exp(1), 
    "discontinuous_1d": lambda x: (x[...,0]) < (1/3),
}

# parameters used for the ridge functions

# The ridge functions:
gs = {
    "jmp": lambda w: w>=1, 
    "knk": lambda w: ((np.minimum(np.maximum(-2,w),1)) + 2) / 3,
    "smo": lambda w: norm.cdf(w + 1),
    "fin": lambda w: np.minimum(1,((np.sqrt(np.maximum(w+2,0)))/2)),
}
d = np.array([1,2,4,16]) # The different d's to test on
ci_methods = np.array(["CLT", "EB", "Betting"]) # The different CI methods

# parameters used for the integrand problems and ridge functions


N_vary = np.array([2**8,2**10,2**12, 2**14, 2**16])# The maximum sample size to be used. Recommended to keep a power of 2 since n must be a power of 2 (QMC rules).
n_vary = 2 ** np.arange(0, 7) # The vector of number of low discrepancy or QMC samples generated per replication
M = 20 # The number of times the computation is repeated

# seed settings

global_seed = 7
parent_seed = np.random.SeedSequence(global_seed)

The function to generate IID replications of QMC samples

In [ ]:
def gen_qmc_samples_iid(discrete_distrib, n = 2**8, function = None, ridge = False):
    x_rld = discrete_distrib.gen_samples(n).reshape((discrete_distrib.replications,n,discrete_distrib.d))
    if ridge is True:
        return x_rld
    if function is None:
        y_rld = x_rld[...,0]
    else:
        y_rld = function(x_rld)
    return y_rld.mean(1),y_rld.flatten()

The function to return the sequence of CLT CI Widths

In [ ]:
def clt_ci_seq (values, times, alpha = 0.05):
    assert np.all(times <= len(values)), f"Invalid values in times: {times[times > len(values)]}"
    ci_arr = np.zeros(len(times))
    for time in range (len(times)):
        curr_val = values[0:times[time]]
        ci_arr[time] = 2 * (t.ppf(1 - alpha / 2,times[time] - 1) * curr_val.std(ddof=1) / np.sqrt(times[time])) 
    return ci_arr

Generating the QMC samples that will be used in both the ridge and integrand functions

In [ ]:
x_qmc_arr = np.empty(len(n_vary), dtype=object)
for i in range(len(n_vary)):
    R_vary = N_vary.max() // n_vary[i]
    child_seed = parent_seed.spawn(1)[0]
    x_qmc_arr[i] = gen_qmc_samples_iid(discrete_distrib=qp.DigitalNetB2(d[-1],seed = child_seed,replications=(M*R_vary)), n = n_vary[i],ridge = True)

# Using Ridge Functions:

In [ ]:
qmc_arr_ridge = np.empty((len(N_vary),M, len(ci_methods),len(d),len(gs), len(n_vary))) # consists of CIs (CLT, EB, Betting) for QMC, the ridge functions, the different dimensions, different R's and n's, and different N's.
for i in range (len(n_vary)):
    x_qmc = norm.ppf(x_qmc_arr[i])
    R_vary = N_vary // n_vary[i]
    for j in range (len(d)):
        w_qmc = x_qmc[:, :, :d[j]].sum(axis = 2)/np.sqrt(d[j])
        m_counter = 0
        for m in range (M):
            m_qmc = w_qmc[m_counter:m_counter + R_vary.max()]
            counter = 0
            for g in gs.values():
                y = g(m_qmc).mean(axis = 1)
                qmc_arr_ridge[:,m,0,j,counter,i] = clt_ci_seq(y,times = R_vary,alpha=alpha) # CLT CI widths
                with suppress_ci_warnings():
                    lower_bound_qmc_integrand_eb,upper_bound_qmc_integrand_eb = predmix_empbern_ci_seq(y, times=R_vary, alpha=alpha, parallel=True, truncation =1/2) 
                # Getting the sequential EB CI widths according to the code from the paper above
                qmc_arr_ridge[:,m,1,j,counter,i] = upper_bound_qmc_integrand_eb - lower_bound_qmc_integrand_eb # The EB CI based on N_vary
                with suppress_ci_warnings():
                    lower_bound_qmc_integrand_bet,upper_bound_qmc_integrand_bet = betting_ci_seq(y, times=R_vary, alpha=alpha, parallel=True, m_trunc=True, trunc_scale=3 / 4) 
                # Getting the sequential Betting CI widths according to the code from the paper above
                qmc_arr_ridge[:,m,2,j,counter,i] = upper_bound_qmc_integrand_bet - lower_bound_qmc_integrand_bet # The Betting CI based on N_vary
                counter = counter + 1
            m_counter = m_counter + R_vary.max()

# The Integrand Problems

In [ ]:
qmc_arr_func = np.empty((len(N_vary),M, len(ci_methods),len(fs), len(n_vary))) # consists of CIs (CLT, EB, Betting) for QMC, the integrands, different R's and n's, and different N's.
for i in range (len(n_vary)):
    R_vary = N_vary // n_vary[i]
    x_qmc = x_qmc_arr[i]
    m_counter = 0
    for m in range (M):
        m_qmc = x_qmc[m_counter:m_counter + R_vary.max()]
        counter = 0
        for f in fs.values():
            y = f(m_qmc).mean(axis = 1)
            qmc_arr_func[:,m,0,counter,i] = clt_ci_seq(y,times = R_vary,alpha=alpha) # CLT CI widths
            with suppress_ci_warnings():
                lower_bound_qmc_integrand_eb,upper_bound_qmc_integrand_eb = predmix_empbern_ci_seq(y, times=R_vary, alpha=alpha, parallel=True, truncation =1/2) 
            # Getting the sequential EB CI widths according to the code from the paper above
            qmc_arr_func[:,m,1,counter,i] = upper_bound_qmc_integrand_eb - lower_bound_qmc_integrand_eb # The EB CI based on N_vary
            with suppress_ci_warnings():
                lower_bound_qmc_integrand_bet,upper_bound_qmc_integrand_bet = betting_ci_seq(y, times=R_vary, alpha=alpha, parallel=True, m_trunc=True, trunc_scale=3 / 4) 
            # Getting the sequential Betting CI widths according to the code from the paper above
            qmc_arr_func[:,m,2,counter,i] = upper_bound_qmc_integrand_bet - lower_bound_qmc_integrand_bet # The Betting CI based on N_vary
            counter = counter + 1
        m_counter = m_counter + R_vary.max()


Appending the data to a csv file:

In [ ]:
rows = []

# Ridge Function Entries
for i, N in enumerate(N_vary):
    for k, n in enumerate(n_vary):
        for g_idx, g_name in enumerate(gs.keys()):
            for d_idx, dim in enumerate(d):
                for ci_idx, ci in enumerate(ci_methods):
                    # Extract the corresponding M values
                    M_values = qmc_arr_ridge[i, :, ci_idx, d_idx, g_idx, k]
                    row = [N, n, g_name, dim, ci] + list(M_values)
                    rows.append(row)

# Integrand Entries 
for i, N in enumerate(N_vary):
    for k, n in enumerate(n_vary):
        for f_idx, f_name in enumerate(fs.keys()):
            for ci_idx, ci in enumerate(ci_methods):
                # Extract the corresponding M values
                M_values = qmc_arr_func[i, :, ci_idx, f_idx, k]
                row = [N, n, f_name, "", ci] + list(M_values)  # Empty string for Dimension
                rows.append(row)

# Define column labels, starting M from M1 instead of M0
col_labels = ["N_vary", "n_vary", "Function", "Dimension", "CI Method"] + [f"M{m+1}" for m in range(M)]

# Create DataFrame
df = pd.DataFrame(rows, columns=col_labels)

# Save to CSV
df.to_csv("qmc_combined_results.csv", index=False)